## MM-SDK

In [ ]:
%store -r
import os,json
from dkube.sdk import *
from dkube.sdk.api import DkubeApi
from dkube.sdk.rsrcs import DkubeModelmonitor
from dkube.sdk.rsrcs.modelmonitor import DkubeModelmonitordataset,DkubeModelmonitoralert
from dkube.sdk.rsrcs.modelmonitor import ModelFrameworks,DatasetClass,AlertClass,ModelType,ModelCategory,DriftAlgo

In [ ]:
api = DkubeApi(URL=os.getenv('DKUBE_URL',DKUBE_URL),token=os.getenv("DKUBE_USER_ACCESS_TOKEN",TOKEN))

In [ ]:
mt=ModelType.Regression
da=DriftAlgo.Auto
dc_t=DatasetClass.TrainData
dc_p=DatasetClass.PredictData
text_file = open("transformer-data.py", "r")
#read whole file to a string
script = text_file.read()
#close file
text_file.close()

### TRAINING

In [ ]:
model_name = 'insurance-model:'+DKUBEUSERNAME
for i in api.get_model_versions(DKUBEUSERNAME,'insurance-model'):
        version=i['version']['uuid']
        model_version = i['version']['name']+':'+version

In [ ]:
training_data = 'insurance-data:'+ DKUBEUSERNAME
train_data_version = 'v1:'+api.get_dataset_versions(DKUBEUSERNAME,'insurance-data')[0]['version']['uuid']

prediction_data = MONITOR_NAME +'-predict:'+ DKUBEUSERNAME
labelled_data = MONITOR_NAME +'-groundtruth:'+ DKUBEUSERNAME

if DATA_SOURCE == 'local':
    predict_data_version = 'v1:'+api.get_dataset_versions(DKUBEUSERNAME,MONITOR_NAME+'-predict')[0]['version']['uuid']
    labelled_data_version = 'v1:'+api.get_dataset_versions(DKUBEUSERNAME,MONITOR_NAME+'-groundtruth')[0]['version']['uuid']

#### 1. CREATE MODEL MONITOR

In [ ]:
mm=DkubeModelmonitor(name=MONITOR_NAME,model_name=model_name,description='mm',tags="d3qatest")
mm.update_modelmonitor(model_type=str(mt),run_freq=5,version=model_version,drift_algo=str(da))

In [ ]:
api.modelmonitor_create(mm,wait_for_completion=True)

#### Extracting ID of the model monitor

In [ ]:
id = api.modelmonitor_get_id(MONITOR_NAME)

#### 2. Add Training Data

In [ ]:
if DATA_SOURCE == 'sql':
    data = DkubeModelmonitordataset(name = "insurance-data-sql:"+DKUBEUSERNAME)
    data.update_dataset(data_class = str(DatasetClass.TrainData),sql_query = "select * from insurance")

else:
    data = DkubeModelmonitordataset(name = training_data)
    data.update_dataset(data_class = str(DatasetClass.TrainData),transformer_script=script,version=train_data_version)

api.modelmonitor_add_dataset(id,data)

#### 3. Update schema

In [ ]:
api.modelmonitor_update_schema(id,label='charges',schema_class='Continuous',schema_type="PredictionOutput",selected=False)
api.modelmonitor_update_schema(id,label='unique_id',schema_class='Continuous',schema_type="RowID",selected=False)
api.modelmonitor_update_schema(id,label='timestamp',schema_class='Continuous',schema_type="Timestamp",selected=False)

## age and bmi to continuous
api.modelmonitor_update_schema(id,label='age',schema_class='Continuous')
api.modelmonitor_update_schema(id,label='bmi',schema_class='Continuous')

## select these features
api.modelmonitor_update_schema(id,label='sex')
api.modelmonitor_update_schema(id,label='children')
api.modelmonitor_update_schema(id,label='smoker')
api.modelmonitor_update_schema(id,label='region')

#### 4. Add Prediction data

In [ ]:
data = DkubeModelmonitordataset(name = prediction_data)
if DATA_SOURCE == "local":
    data.update_dataset(data_class = str(DatasetClass.PredictData),version = predict_data_version)
if DATA_SOURCE == "aws_s3":
    data.update_dataset(data_class = str(DatasetClass.PredictData))
if DATA_SOURCE == "sql":
    data = DkubeModelmonitordataset(name = "insurance-data-sql:"+DKUBEUSERNAME)
    data.update_dataset(data_class = str(DatasetClass.PredictData),sql_query = "select * from insurance_predict")

api.modelmonitor_add_dataset(id,data)

#### 4. Add Labelled data

In [ ]:
data = DkubeModelmonitordataset(name =labelled_data)
if DATA_SOURCE == 'local':
    data.update_dataset(data_class = str(DatasetClass.LabelledData),groundtruth_col='GT_target',predict_col='charges',version=labelled_data_version)
if DATA_SOURCE == 'aws_s3':
    data.update_dataset(data_class = str(DatasetClass.LabelledData),groundtruth_col='GT_target',predict_col='charges')
if DATA_SOURCE == "sql":
    data = DkubeModelmonitordataset(name = "insurance-data-sql:"+DKUBEUSERNAME)
    data.update_dataset(data_class = str(DatasetClass.LabelledData),groundtruth_col='GT_target',predict_col='charges',sql_query="select * from insurance_gt")

api.modelmonitor_add_dataset(id,data)

#### 5. Add Alert

#### Feature Alerts

In [ ]:
alert = DkubeModelmonitoralert(name='age_alert')
alert.update_alert(alert_class = 'FeatureDrift',feature='age',threshold=0.02)
api.modelmonitor_add_alert(id,alert)

In [ ]:
alert = DkubeModelmonitoralert(name='bmi_alert')
alert.update_alert(alert_class = 'FeatureDrift',feature='bmi',threshold=0.05)
api.modelmonitor_add_alert(id,alert)

#### Performance Alerts

In [ ]:
alert = DkubeModelmonitoralert(name='metrics_alert')
alert.update_alert(alert_class = 'PerformanceDecay',metric = 'mse',threshold=36000000)
api.modelmonitor_add_alert(id,alert)

#### 6. Start the Model Monitor

In [ ]:
api.modelmonitor_start(id)

### 7. RETRAINING

In [ ]:
%store -r
id = api.modelmonitor_get_id(MONITOR_NAME)

if INPUT_TRAIN_TYPE == 'retraining':
    api.modelmonitor_stop(id)
    id = api.modelmonitor_get_id(MONITOR_NAME)
    model_name = 'insurance-model:'+ DKUBEUSERNAME
    models = api.get_model_versions(DKUBEUSERNAME,'insurance-model')[0]['version']
    model_version = models['name']+":"+models['uuid']
    
    ## update model version 
    mm=DkubeModelmonitor(MONITOR_NAME,model_name=model_name,description="mm")
    mm.update_model_version(model_version)
    api.modelmonitor_update(id,mm)
    

    training_data = 'insurance-training-data:'+DKUBEUSERNAME
    data_dict = api.get_dataset_versions(DKUBEUSERNAME,'insurance-training-data')[0]['version']
    train_data_version = data_dict['name']+":"+data_dict['uuid']
    
    ### update training dataset version
    data = DkubeModelmonitordataset(name=training_data)
    data.update_dataset(version=train_data_version)
    api.modelmonitor_update_dataset(id=id,data_class=str(DatasetClass.TrainData),data=data)
    
    
    ### Start the model monitor
    api.modelmonitor_start(id)

### 8. Cleanup

In [ ]:
CLEANUP = False
if CLEANUP:
    api.modelmonitor_delete(id)
    